In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.color import lab2rgb
import sys
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from skimage.color import rgb2lab
from sklearn.pipeline import make_pipeline


# representative RGB colours for each label, for nice display
COLOUR_RGB = {
    'red': (255, 0, 0),
    'orange': (255, 114, 0),
    'yellow': (255, 255, 0),
    'green': (0, 230, 0),
    'blue': (0, 0, 255),
    'purple': (187, 0, 187),
    'brown': (117, 60, 0),
    'black': (0, 0, 0),
    'grey': (150, 150, 150),
    'white': (255, 255, 255),
}
name_to_rgb = np.vectorize(COLOUR_RGB.get, otypes=[np.uint8, np.uint8, np.uint8])


def plot_predictions(model, lum=71, resolution=256):
    """
    Create a slice of LAB colour space with given luminance; predict with the model; plot the results.
    """
    wid = resolution
    hei = resolution

    # create a hei*wid grid of LAB colour values, with L=lum
    ag = np.linspace(-100, 100, wid)
    bg = np.linspace(-100, 100, hei)
    aa, bb = np.meshgrid(ag, bg)
    ll = lum * np.ones((hei, wid))
    lab_grid = np.stack([ll, aa, bb], axis=2)

    # convert to RGB for consistency with original input
    X_grid = lab2rgb(lab_grid)

    # predict and convert predictions to colours so we can see what's happening
    y_grid = model.predict(X_grid.reshape((wid*hei, 3)))
    pixels = np.stack(name_to_rgb(y_grid), axis=1) / 255
    pixels = pixels.reshape((hei, wid, 3))

    # plot input and predictions
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title('Inputs')
    plt.imshow(X_grid.reshape((hei, wid, 3)))

    plt.subplot(1, 2, 2)
    plt.title('Predicted Labels')
    plt.imshow(pixels)


In [2]:
data = pd.read_csv("colour-data.csv")

In [3]:
X = (data[['R','G','B']]/255).values
X.shape

(2429, 3)

In [4]:
y = data['Label'].values
y.shape

(2429,)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
model = GaussianNB()
model.fit(X_train, y_train)
# print(model.theta_)  # sample means
# print(model.sigma_) # sample variances
y_predicted = model.predict(X_test)
print(accuracy_score(y_test, y_predicted))

0.59375


In [7]:
def getLabColor(X):
    X2d = X[:, np.newaxis]
    X3d = X2d[:, np.newaxis]
    XLab3d = rgb2lab(X3d)
    return np.reshape(XLab3d, (len(X), 3))

In [8]:
# lab = getLabColor(X_train)

In [9]:
from sklearn.preprocessing import FunctionTransformer
pipeline = make_pipeline(
        FunctionTransformer(getLabColor),
        GaussianNB()
    )

In [13]:
pipeline.fit(X_train, y_train)
y_predictedLab = pipeline.predict(X_test)
print(accuracy_score(y_test, y_predictedLab))

0.629934210526
